## generate narrative from resource

### import modules

In [ ]:
import os, sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
# This allows you to import the desired function from the module hierarchy:
from fhirclient.r4models.fhirabstractbase import FHIRValidationError
from fhirclient.r4models import capabilitystatement as CS
from fhirclient.r4models import narrative as N
from fhirclient.r4models import meta as M
import fhirclient.r4models.fhirdate as D
from json import dumps, loads, load, JSONDecodeError
from datetime import datetime, date
from requests import get, post, put
from pathlib import Path
from IPython.display import display as Display, HTML, Markdown
from pprint import pprint
from collections import namedtuple
from jinja2 import Environment, FileSystemLoader, select_autoescape
from stringcase import snakecase, titlecase
from itertools import zip_longest
from commonmark import commonmark
from lxml import etree
import tarfile

## Functions

### Save updated CS to local dir

In [ ]:
def save_me(cs_py):
    ig_source_path = ''
    #save in ig_source folder
    path = Path.cwd() / ig_source_path / 'resources' / f'capabilitystatement-{cs_py.id.lower()}.json'
    print(f'...........saving to file {path}............')
    path.write_text(dumps(cs_py.as_json(), indent=4))

### Get spec_internals from package file from local directory and return as dict

In [ ]:
def get_si(tar_file):
    print(path)
    with tarfile.open(tar_file, mode='r') as tf:
        #pprint(tf.getnames())
        f = tf.extractfile('package/other/spec.internals')
        r = f.read()
        return(loads(r))

### Get Name/Title from resource

In [ ]:
def get_name_map(p_file,r_dict):
    try:
        k = r_dict['url']
        v = r_dict['title']
        return({k:v})
    except KeyError:
        try:
            k = r_dict['url']
            v = r_dict['name']
            return({k:v})
        except KeyError: # not a conf resource
            print(f'****{p_file} is not a conf resource***')   

### Get SP/Type from resource

In [ ]:
def get_sp_map(p_file,r_dict):
    try:
        if r_dict["resourceType"]=='SearchParameter':
            k = r_dict['code']
            v = r_dict['type']
            print(r_dict['type'])
            return({k:v})
    except KeyError:
        print(f'{p_file} has no resourceType or code or type')    

### Get CS

In [ ]:
def get_cs_map(p_file,r_dict):
    try:
        if r_dict["resourceType"]=='CapabilityStatement':
            k = p_file.split('/')[-1]
            v = r_dict
            return({k:v})
    except KeyError:
        print(f'{p_file} has no resourceType')    

### Get package files and map based on type

In [ ]:
def get_my_map(tar_file,map_type):
    print(path)
    my_map = {}
    with tarfile.open(tar_file, mode='r') as tf:
        for p_file in tf.getnames():
            #print(p_file)
            f = tf.extractfile(p_file)
            r = f.read()
            #print(type(r))
            try:
                r_dict = loads(r)
            except JSONDecodeError: # not a json file
                continue
            if map_type == 'name_map':
                try:
                    my_map.update(get_name_map(p_file,r_dict))
                except TypeError:
                    pass
            elif map_type == 'sp_map':
                try:
                    my_map.update(get_sp_map(p_file,r_dict)) 
                except TypeError:
                    pass
            elif map_type == 'cs_map':
                try:
                    my_map.update(get_cs_map(p_file,r_dict)) 
                except TypeError:
                    pass
    return(my_map)

## Fetch package file
- Get spec_internal from package a json file which includes canonical to local relative page links

(todo add a graphic to show where to find this file - could eliminate above step too...)

In [ ]:
ig_package = input("Get NPM package url from the IG Publication (Version) History page:")

### Fetch package file from url and save to local file

In [ ]:
r = get(ig_package)
print(f'HTTP Status: {r.status_code}')
path = Path.cwd() /  'cs_source_file/package.tgz'
print(f'saving to: {path}')
path.write_bytes(r.content)

### Get spec internals from package file and create a list

In [ ]:
si = get_si(path)
       
path_map = si['paths']
path_map

### Get URL:Name/Title mapping from package file

In [ ]:
name_map = get_my_map(path,map_type='name_map')
name_map   

### Get SearchParameter parameter name: type mapping from package file

In [ ]:
sp_map = get_my_map(path,map_type='sp_map')
sp_map

## Get CapStatements from package and iterate through and create narratives

### Get CapStatements as dict name = name of file, value = resource

In [ ]:
cs_dict = get_my_map(path,map_type='cs_map')
cs_dict

### iterate through the dictionary of CapabilityStatements render using Jinja2 templates and save the files

In [ ]:
in_path = ''
in_file = 'R4capabilitystatement-server.j2'

def markdown(text, *args, **kwargs):
    return commonmark(text, *args, **kwargs)

env = Environment(
    loader=FileSystemLoader(searchpath = in_path),
    autoescape=select_autoescape(['html','xml','xhtml','j2','md'])
    )

env.filters['markdown'] = markdown

template = env.get_template(in_file)

for k,v in cs_dict.items():
    cs_py = CS.CapabilityStatement(v) # create CS instance
    rendered = template.render(cs=cs_py,
                               path_map=path_map,
                               pname_map=name_map,
                               sp_map=sp_map )

    display(HTML(rendered))

    parser = etree.XMLParser(remove_blank_text=True)
    root = etree.fromstring(rendered, parser=parser)

    div = (etree.tostring(root[1][0], encoding='unicode', method='html'))
    narr = N.Narrative()
    meta = M.Meta()
    meta.lastUpdated = D.FHIRDate(str(date.today()))
    cs_py.meta = meta
    print(cs_py.meta.as_json())
    narr.status = 'generated'
    narr.div = div
    cs_py.text = narr
    


    #print(dumps(cs.as_json(),indent=3))
    #validate_me(cs_py)
    save_me(cs_py)